In [2]:
import ccxt
import ta
import pandas as pd
import numpy as np

# Initialize the Binance exchange
exchange = ccxt.binance()

# Retrieve the daily and 4h candlestick data for the ETH/USDT trading pair from Binance
ohlcv_daily = exchange.fetch_ohlcv('ETH/USDT', timeframe='1d')
ohlcv_4h = exchange.fetch_ohlcv('ETH/USDT', timeframe='4h')

# Convert the candlestick data to a pandas DataFrame
df_daily = pd.DataFrame(ohlcv_daily, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df_4h = pd.DataFrame(ohlcv_4h, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

# Convert the timestamp to a datetime object
df_daily['timestamp'] = pd.to_datetime(df_daily['timestamp'], unit='ms')
df_4h['timestamp'] = pd.to_datetime(df_4h['timestamp'], unit='ms')

# Set the timestamp as the index
df_daily.set_index('timestamp', inplace=True)
df_4h.set_index('timestamp', inplace=True)

# Calculate the fixed range volume profile for the last seven days and the last 14 days on the daily timeframe
vp_daily_7d = ta.volume.VolumeProfile(price_data=df_daily['close'], num_of_bins=100, window=7)
vp_daily_14d = ta.volume.VolumeProfile(price_data=df_daily['close'], num_of_bins=100, window=14)

# Get the highest peak of the volume profile within the last three candles
last_3_candles = df_daily.iloc[-3:]
vp_7d_last_3_candles = vp_daily_7d.get_distribution(last_3_candles['close'])
vp_14d_last_3_candles = vp_daily_14d.get_distribution(last_3_candles['close'])
highest_peak_7d = ta.utils.get_a_largest_peak(vp_7d_last_3_candles)
highest_peak_14d = ta.utils.get_a_largest_peak(vp_14d_last_3_candles)

# Determine if the current price is over the fixed range volume profile of the last seven days and the last 14 days in the last three candles
current_price = df_daily['close'][-1]
if current_price > highest_peak_7d and current_price > highest_peak_14d:
    # Go long
    entry_price = current_price
    stop_loss = np.min(last_3_candles[last_3_candles['close'] > highest_peak_7d]['low'])

# Calculate the RSI on the daily timeframe
rsi_period = 14
df_daily['rsi'] = ta.momentum.RSIIndicator(df_daily['close'], rsi_period).rsi()

# Monitor the price and RSI on the daily timeframe
while True:
    # Check if the price makes a higher high while the RSI makes a lower high
    highest_high = df_daily['high'].max()
    highest_high_index = df_daily['high'].idxmax()
    highest_high_rsi = df_daily['rsi'][highest_high_index]
    if highest_high > entry_price and highest_high_rsi < df_daily['rsi'].max():
        # Take profit
        take_profit = highest_high
        break
    
    # Check if the stop loss is hit
    if df_daily['low'][-1] < stop_loss:
        # Exit the trade
        break
    
    # Wait for the next candle on the daily timeframe
    exchange.sleep(exchange.rateLimit / 1000)
    df_daily_new = exchange.fetch_ohlcv('ETH/USDT', timeframe='1d')
    df_daily_new = pd.DataFrame(df_daily_new, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df_daily_new['timestamp'] = pd.to_datetime(df_daily_new['timestamp'], unit='ms')
    df_daily_new.set_index('timestamp', inplace=True)
    df_daily = df_daily.append(df_daily_new)
    df_daily['rsi'] = ta.momentum.RSIIndicator(df_daily['close'], rsi_period).rsi()

# Exit the trade
if take_profit is not None:
    profit = take_profit - entry_price
    print(f"Take profit hit! Profit: {profit:.2f}")
else:
    loss = entry_price - stop_loss
    print(f"Stop loss hit! Loss: {loss:.2f}")


AttributeError: module 'ta.volume' has no attribute 'VolumeProfile'